In [1]:
import json
import pickle
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
from sklearn.preprocessing import LabelEncoder
from pprint import pprint
from utils import *

tagger = MeCab.Tagger()

D:\anaconda3\envs\data_analysis\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
input_data_path = '../data/input_data/'
output_data_path = '../data/output_data/'
outer_data_path = '../data/outer_data/'

In [3]:
with open(input_data_path+'train_aozora_data.json') as f:
    novels_json = json.load(f)

novel_id_l = []
author_l = []
for author_data in novels_json:
    for novel in author_data['novels']:
        novel_id_l.append(novel['content_id'])
        author_l.append(author_data['author_id'])

In [4]:
model_path = '../data/outer_data/chive-1.2-mc5.kv'
w2v_model = KeyedVectors.load(model_path)

In [5]:
with open(input_data_path+'aozora_contents.json', encoding='utf-8') as f:
    novel_text_dic = json.load(f)

test_l = []
i = 0
for novel_content_id, author_id in zip(novel_id_l, author_l):
    novel_content = novel_text_dic[novel_content_id].replace('\n', '').replace('\r', '').replace('\u3000', '')
    novel_word_l = doc_wakati(novel_content)
    text_vec = text_to_vec(novel_word_l, w2v_model)

    if text_vec is None:
        continue

    test_l.append([author_id] + list(text_vec))
    print('\r'+str(i), end='')
    i+=1

train_df = pd.DataFrame(test_l, columns=['author_id'] + [i for i in range(300)])

13861

In [6]:
with open(input_data_path+'aozora_authors.json', encoding='utf-8') as f:
    authors_dict = json.load(f)
le = LabelEncoder()
le.fit(list(authors_dict.keys()))
train_df['author_id'] = le.transform(train_df['author_id'])

In [7]:
with open('../data/output_data/label_encoder.pickle', 'wb') as f:
    pickle.dump(le, f)

In [8]:
with open(output_data_path+'train_df.pickle', 'wb') as f:
    pickle.dump(train_df, f)

In [9]:
train_df

,author_id,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
0,0,-112.133827,-109.762787,465.392151,-157.924301,-16.924057,591.945618,361.975250,-126.534622,339.810089,...,-38.189274,192.175400,-184.934235,357.455811,17.801519,-84.512581,9.809142,-116.553688,12.258219,312.771667
1,0,-172.398926,-224.391296,796.502441,-173.684296,-16.879339,1003.801514,565.185669,-221.078735,550.393921,...,-73.855629,276.672455,-302.772125,585.884460,55.069267,-99.017212,-35.038902,-223.876602,42.310093,512.034729
2,0,-67.706779,-164.924301,544.071472,-227.151947,-71.958115,770.307739,583.036560,-237.669922,477.266296,...,-49.152954,221.674881,-289.982941,489.691498,33.718834,-114.547760,-29.271347,-167.962677,94.906235,414.354126
3,2,-102.873665,-259.469513,520.554871,-275.368561,-99.724724,846.361633,581.307739,-217.451340,431.908600,...,-129.024963,284.965881,-306.776428,447.656036,93.829048,-78.314171,-62.068214,-83.681923,79.618904,403.603912
4,3,-26.612259,-76.835815,237.728607,-43.423389,-14.492610,283.665985,194.037399,-69.267273,166.528641,...,-38.023846,58.870335,-80.066513,155.247681,49.369865,-28.532068,-7.371575,-83.570236,9.611092,148.603287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13857,868,-39.061272,-33.194981,124.216911,-31.266853,-27.426500,143.982056,92.168488,-30.151125,84.128784,...,-18.688509,12.885788,-34.294350,65.770821,4.551261,-20.840319,-3.647400,-28.524019,12.114426,76.176025
13858,868,-75.441772,-77.858604,264.147980,-160.411392,-81.281029,376.551758,332.840729,-96.692841,251.082596,...,-97.681908,112.585442,-147.852798,185.304108,56.416836,14.123760,-23.609854,-94.893280,-37.999367,252.542221
13859,868,-60.808270,-78.430458,267.261475,-150.407120,-77.048515,384.530853,324.920105,-128.278839,296.096008,...,-133.577789,123.531761,-175.936523,205.229004,18.520966,-28.218449,-56.361561,-36.037376,-29.129564,220.083252
13860,868,-42.557716,-325.378601,852.216248,-192.011475,-34.691586,1149.957764,760.387451,-211.066406,561.837463,...,-176.986771,374.603271,-444.007416,529.182861,46.357437,-23.589565,-73.779083,-225.363708,-7.610521,458.546967


In [14]:
#ここからテストデータの前処理
with open(input_data_path+'test_q.json', encoding='utf-8') as f:
    test_json = json.load(f)
with open(input_data_path+'test_aozora_data.json', encoding='utf-8')as f:
    title2con_dict = json.load(f)

candis_list = []
test_l = []
for test_data in test_json:
    candis_list.append(le.transform(test_data['candidates']))
    novel_content_id = title2con_dict[test_data['title_id']]
    novel_content = novel_text_dic[novel_content_id].replace('\n', '').replace('\r', '').replace('\u3000', '')
    novel_word_l = doc_wakati(novel_content)
    text_vec = text_to_vec(novel_word_l, w2v_model)

    if text_vec is None:
        test_l.append([0 for _ in range(300)])
        continue

    test_l.append(list(text_vec))

columns = pd.Index([i for i in range(300)], dtype='object')
test_df = pd.DataFrame(test_l, columns=columns)
test_df

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-34.749607,-217.841919,492.708710,-150.425369,56.124329,609.579529,492.638824,-98.105240,457.823120,-336.838074,...,-88.017319,136.950333,-183.918427,404.317444,2.175942,-109.304703,37.118591,-188.553864,44.708824,397.738556
1,-69.822678,-63.136490,351.433258,2.645442,25.182114,277.310394,91.766357,-44.883926,170.521774,-144.073669,...,-33.936909,72.949760,-156.965805,233.196945,-71.126328,4.324072,5.187243,-29.796396,114.808472,255.200150
2,-48.095352,-46.986305,499.996277,-224.369156,-44.976654,654.668091,530.025452,-56.252655,441.355194,-337.981659,...,-70.220741,182.682587,-194.136826,368.106476,85.588051,-28.405197,29.644566,-129.618530,45.164185,270.153870
3,-122.358849,-197.311768,835.409912,-248.343643,-38.418217,1024.928589,529.685242,-185.226089,499.097534,-481.770050,...,-32.542843,276.022583,-284.785828,535.537109,20.867434,-173.357208,57.438450,-268.337738,37.144539,524.585754
4,-132.708054,-182.505478,624.345581,-189.538513,-27.550827,897.433228,608.874817,-187.248169,471.478577,-420.250793,...,-58.654812,235.929443,-275.715912,476.217072,71.522057,-85.749290,-31.858097,-181.736908,35.227646,461.789825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543,-30.212027,-62.102764,229.267288,-65.241951,-25.118748,340.062500,280.279114,-132.817459,209.913498,-212.883636,...,-30.571115,115.062340,-109.842415,185.753174,33.738625,-34.878052,21.567463,-89.767921,19.864609,144.914398
544,-26.898439,-68.969589,243.487381,-82.365669,10.187095,279.653931,229.362671,-56.927586,183.669617,-166.005035,...,-24.230587,96.883347,-87.069756,169.617981,-28.779482,-33.748997,-12.120331,-70.996719,38.664112,140.356262
545,-28.189156,-35.155735,125.416290,-15.487114,6.063377,125.886749,51.905651,-17.445158,78.082024,-73.932556,...,-3.629410,28.858280,-63.481209,90.364395,-8.135536,-3.711974,-3.844317,-24.210222,12.298188,77.785278
546,-94.718971,-127.314781,524.823547,-53.116463,38.460175,497.531921,177.434708,-121.534073,276.677338,-344.934509,...,8.480258,146.624191,-218.897415,344.743500,10.261924,-15.420820,-0.986041,-18.372000,76.771812,385.276703


In [15]:
# with open(output_data_path+'test_df.pickle', 'wb') as f:
#     pickle.dump(test_df, f)
# with open(output_data_path+'test_df.pickle', 'rb') as f:
#     test_df = pickle.load(f)

with open(output_data_path+'candis_list.pickle', 'wb') as f:
    pickle.dump(list(map(list, candis_list)), f)

test_df.to_pickle(output_data_path+'test_df.pickle')